In [1]:
import pandas as pd
import time

In [2]:
t1 = time.time()
for i in range(2019, 2022) :
    
    caracteristiques1 = 'caracteristiques-' + str(i) + '.csv'
    lieux1 = 'lieux-' + str(i) + '.csv'
    usagers1 = 'usagers-' + str(i) + '.csv'
    vehicules1 = 'vehicules-' + str(i) + '.csv'

    caracteristiques = pd.read_csv(caracteristiques1 , sep=";")
    lieux = pd.read_csv(lieux1, sep=";")
    usagers = pd.read_csv(usagers1, sep=";")
    vehicules = pd.read_csv(vehicules1, sep=";")
        

    df1 = caracteristiques.merge(lieux, how = 'outer')

    df2 = usagers.merge(vehicules, how = 'outer')

    df2 = df2.groupby('Num_Acc', group_keys=False).apply(lambda x: x.loc[x.grav.idxmax()])

    df2 = df2.reset_index(drop=True)

    df_final = df1.merge(df2, how = 'outer')
    
    if i == 2019 :
        df_inter = df_final.copy()
    else : 
        df = pd.concat([df_final , df_inter], ignore_index = True) 
        if i !=2021 :
            df_inter = df.copy()   
t2 = time.time()
print(t2-t1)

209.31698083877563


In [3]:
df.shape

(163102, 54)

#### On retire les départements d'outre mer et on concentre sur la france metropolitaine pour le reste de travail

In [4]:
list_outre_mer = [ '971','972','973','974','975','976','977','978','986','987','988']
    
for i in list_outre_mer :
    df.drop( df[ df['dep'] == i ].index, inplace=True)
    
df.shape

(154677, 54)

#### Ajout des noms de région

In [5]:
df_region = pd.read_csv('departements-france.csv')
df_region.rename(columns={'code_departement': 'dep'},inplace=True)
df = pd.merge(df, df_region, on ='dep', how ='left')

In [6]:
df.shape

(154677, 57)

#### Créaction de la colonne Date Accident

In [7]:
#Changement du type des colonnes pour effectuer les changement
df.jour = df.jour.astype(str)
df.mois = df.mois.astype(str)
df.hrmn = df.hrmn.astype(str)
df.an = df.an.astype(str)
df.dep = df.dep.astype(str)

df['Date Accident'] = ['-'.join(i) for i in zip(df['an'], df['mois'], df['jour'])]
df['Date Accident'] = [' '.join(i) for i in zip(df['Date Accident'], df['hrmn'])]

df['Date Accident'] = pd.to_datetime(df['Date Accident'])

#### Téléchargement de la base finale

In [10]:
df.to_csv('df_final.csv', index=False, sep=';')

In [16]:
df.groupby(['grav']).count()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,obs,obsm,choc,manv,motor,occutc,nom_departement,code_region,nom_region,Date Accident
grav,,,,,,,,,,,,,,,,,,,,,
2.0,5744,5744,5744,5744,5744,5744,5744,5744,5744,5744,...,5744,5744,5744,5744,5744,50,5590,5590,5590,5744
3.0,38868,38868,38868,38868,38868,38868,38868,38868,38868,38868,...,38868,38868,38868,38868,38868,167,37746,37746,37746,38868
4.0,110065,110065,110065,110065,110065,110065,110065,110065,110065,110065,...,110065,110065,110065,110065,110065,841,108375,108375,108375,110065
